# ASSIGNMENT: Cartoon Face Mask
## This assignment tests your ability to apply face recognition, image masks, image thresholding, video capturing and saving video feed into a video file concepts.

## TASK: Cartoonify faces in video feed from live webcam

### Steps
- 1. **Capture video** feed from webcam
- 2. **Recognize faces** in the video
- 3. **Replace/Mask the face** region with your favorite cartoon character
- 4. **Save the video** feed into a video file
- 5. Submit the following files
    - Notebook (.ipynb)
    - Video (.avi/.mp4)

### Helper code to recognize faces

In [5]:
import cv2
import numpy as np
import os

### Read image to replace face

In [6]:
img= cv2.imread('miku.png')
#cv2.imshow('Image', img)
#cv2.waitKey(0)
#resize image because it´s too big
#r = 200.0 /img.shape[1]
#dim = (200, int(img.shape[0] * r))
#small = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
#cv2.imshow('Normal Image' , np.hstack([small]))
#cv2.waitKey(0)

In [9]:
def save_webcam(outPath,fps,mirror=False):
    cascPathface = os.path.dirname(cv2.__file__) + "/data/haarcascade_frontalface_alt2.xml"
    cascPatheyes = os.path.dirname(cv2.__file__) + "/data/haarcascade_eye_tree_eyeglasses.xml"
    
    faceCascade = cv2.CascadeClassifier(cascPathface)
    eyeCascade = cv2.CascadeClassifier(cascPatheyes)
    
    video_capture = cv2.VideoCapture(0)
    currentFrame = 0
    
    # Get current width of frame
    width = video_capture.get(cv2.CAP_PROP_FRAME_WIDTH)  # float
    # Get current height of frame
    height = video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT)  # float
    
    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'DIVX')
    out = cv2.VideoWriter(outPath, fourcc, fps, (int(width), int(height)))
    
    while True:
        # Capture frame-by-frame
        ret, frame = video_capture.read()
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = faceCascade.detectMultiScale(gray,
                                             scaleFactor=1.1,
                                             minNeighbors=5,
                                             minSize=(60, 60),
                                             flags=cv2.CASCADE_SCALE_IMAGE)
        for (x,y,w,h) in faces:
            cv2.rectangle(frame, (x, y), (x + w, y + h),(0,255,0), 2)
            faceROI = frame[y:y+h,x:x+w]
            eyes = eyeCascade.detectMultiScale(faceROI)

            
            img3 = frame.copy()
            #resize image because it´s too big
            r = (y+h) /img.shape[1]
            dim = (x+w,int(img.shape[1] * r))
            resized = cv2.resize(img,faceROI.shape[:2], interpolation = cv2.INTER_AREA)
            cv2.imshow('MIku', resized)
            
            x_offset=x
            y_offset=y
            
            #########THIS LINE IT´S FOR SHOW THE WHOLE IMAGE WITOUTH ALPHA
            #img3[y_offset:y_offset+resized.shape[0], x_offset:x_offset+resized.shape[1]] = resized
            
            
            #######THIS PART IT´S FOR SHOW IMAGE WITH TRANSPARENCY (LOOKS BETTER THIS WAY)
            y1, y2 = y_offset, y_offset + resized.shape[0]
            x1, x2 = x_offset, x_offset + resized.shape[1]
            alpha_s = resized[:, :, 2] / 255.0
            alpha_l = 0.4 - alpha_s
            
            for c in range(0, 3):
                img3[y1:y2, x1:x2, c] = (alpha_s * resized[:, :, c] +
                              alpha_l * img3[y1:y2, x1:x2, c])
            
            for (x2, y2, w2, h2) in eyes:
                eye_center = (x + x2 + w2 // 2, y + y2 + h2 // 2)
                radius = int(round((w2 + h2) * 0.25))
                frame = cv2.circle(frame, eye_center, radius, (255, 0, 0), 4)
    
            # Display the resulting frame
            cv2.imshow('Face Video', img3)
            cv2.imshow("Face ROI", faceROI)
            
        # Saves for video
        #out.write(img3)
        # To stop duplicate images
        currentFrame += 1
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    video_capture.release()
    out.release()
    cv2.destroyAllWindows()

In [10]:
def main():
    save_webcam('output.mp4', 20.0,mirror=True)

if __name__ == '__main__':
    main()